In [1]:
from google.colab import drive

# Liên kết Google Drive với Google Colab
drive.mount('/content/drive')

# Sau khi thực hiện đoạn mã trên, Drive của bạn sẽ được gắn vào đường dẫn /content/drive/
# Bạn có thể truy cập các file trong Google Drive thông qua đường dẫn này.


Mounted at /content/drive


In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import sent_tokenize

# Ensure you have NLTK tokenizer downloaded
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader
from torch import cuda
import sys
from sklearn.metrics import f1_score
import csv


In [12]:
model_path = '/content/drive/MyDrive/3c-citation-text-classification/Task 1/best_model.pt'  # Đường dẫn đến model đã lưu
model_name = 'allenai/scibert_scivocab_uncased'
drop_out = 0.1


validation_file = '/content/drive/MyDrive/3c-citation-text-classification/scisum/data/train_citation.csv'  # Tệp dữ liệu đầu vào
output_file = '/content/drive/MyDrive/3c-citation-text-classification/scisum/data/predictions.csv'  # Tệp kết quả đầu ra


In [5]:

LMTokenizer = AutoTokenizer.from_pretrained(model_name)
LMModel = AutoModel.from_pretrained(model_name)

device = 'cuda' if cuda.is_available() else 'cpu'


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [17]:
# Định nghĩa lớp mô hình
class LMClass(torch.nn.Module):
    def __init__(self):
        super(LMClass, self).__init__()
        self.l1 = LMModel
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(drop_out)
        self.classifier = torch.nn.Linear(768, 6)

    def forward(self, data):
        input_ids_cc = data['CC_ids'].to(device, dtype=torch.long)
        attention_mask_cc = data['CC_mask'].to(device, dtype=torch.long)

        output_cc = self.l1(input_ids=input_ids_cc, attention_mask=attention_mask_cc)
        hidden_state_cc = output_cc[0][:, 0]

        pooler = self.pre_classifier(hidden_state_cc)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

'''
# Định nghĩa lớp mô hình
class LMClass(torch.nn.Module):
    def __init__(self):
        super(LMClass, self).__init__()
        self.l1 = LMModel
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(drop_out)
        self.classifier = torch.nn.Linear(768, 6)

    def forward(self, data):
        input_ids_cgt = data['CGT_ids'].to(device, dtype=torch.long)
        attention_mask_cgt = data['CGT_mask'].to(device, dtype=torch.long)

        input_ids_cdt = data['CDT_ids'].to(device, dtype=torch.long)
        attention_mask_cdt = data['CDT_mask'].to(device, dtype=torch.long)

        input_ids_cc = data['CC_ids'].to(device, dtype=torch.long)
        attention_mask_cc = data['CC_mask'].to(device, dtype=torch.long)

        output_cgt = self.l1(input_ids=input_ids_cgt, attention_mask=attention_mask_cgt)
        output_cdt = self.l1(input_ids=input_ids_cdt, attention_mask=attention_mask_cdt)
        output_cc = self.l1(input_ids=input_ids_cc, attention_mask=attention_mask_cc)

        # Lấy hidden state của các đầu vào
        hidden_state_cgt = output_cgt[0][:, 0]
        hidden_state_cdt = output_cdt[0][:, 0]
        hidden_state_cc = output_cc[0][:, 0]

        # Tổng hợp lại các đặc trưng từ CGT, CDT, CC
        combined_hidden = hidden_state_cgt + hidden_state_cdt + hidden_state_cc

        pooler = self.pre_classifier(combined_hidden)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
'''

"\n# Định nghĩa lớp mô hình\nclass LMClass(torch.nn.Module):\n    def __init__(self):\n        super(LMClass, self).__init__()\n        self.l1 = LMModel\n        self.pre_classifier = torch.nn.Linear(768, 768)\n        self.dropout = torch.nn.Dropout(drop_out)\n        self.classifier = torch.nn.Linear(768, 6)\n\n    def forward(self, data):\n        input_ids_cgt = data['CGT_ids'].to(device, dtype=torch.long)\n        attention_mask_cgt = data['CGT_mask'].to(device, dtype=torch.long)\n\n        input_ids_cdt = data['CDT_ids'].to(device, dtype=torch.long)\n        attention_mask_cdt = data['CDT_mask'].to(device, dtype=torch.long)\n\n        input_ids_cc = data['CC_ids'].to(device, dtype=torch.long)\n        attention_mask_cc = data['CC_mask'].to(device, dtype=torch.long)\n\n        output_cgt = self.l1(input_ids=input_ids_cgt, attention_mask=attention_mask_cgt)\n        output_cdt = self.l1(input_ids=input_ids_cdt, attention_mask=attention_mask_cdt)\n        output_cc = self.l1(input_

In [18]:
# Khởi tạo lớp dự đoán
class LMClassPredictor:
    def __init__(self, model_path, tokenizer, device='cpu'):
        self.device = device
        self.model = LMClass()
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.to(device)
        self.model.eval()
        self.tokenizer = tokenizer

    def predict(self, cc):
        inputs_cc = self.tokenizer.encode_plus(
            cc,
            None,
            add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )

        data = {
            'CC_ids': torch.tensor(inputs_cc['input_ids'], dtype=torch.long).unsqueeze(0).to(self.device),
            'CC_mask': torch.tensor(inputs_cc['attention_mask'], dtype=torch.long).unsqueeze(0).to(self.device),
        }

        with torch.no_grad():
            outputs = self.model(data)
            probabilities = torch.softmax(outputs, dim=1).squeeze()
            predicted_labels = torch.argmax(probabilities).item()

        return predicted_labels, probabilities

In [21]:
predictor = LMClassPredictor(model_path= model_path, tokenizer=LMTokenizer, device=device)

# Đọc dữ liệu từ tập tin validation
validation_data = pd.read_csv(validation_file, sep=',', names=['CGT', 'CDT', 'CC', 'label'])

<ipython-input-18-91373e22723d>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_location=device))


In [23]:
# Dự đoán và lưu kết quả vào CSV
with open(output_file, 'w', newline='') as csvfile:
    # Định nghĩa tên cột trong file CSV
    fieldnames = ['CGT', 'CDT', 'CC', 'Predicted_Label', 'label 0', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for index, row in validation_data.iterrows():
        cgt = row['CGT']
        cdt = row['CDT']
        cc = row['CC']  # Lấy dữ liệu từ cột thứ 2 (CC)
        predicted_label, probabilities = predictor.predict(cc)
        writer.writerow({
            'CGT': cgt,
            'CDT': cdt,
            'CC': cc,
            'Predicted_Label': predicted_label,
            'label 0': probabilities[0],
            'label 1': probabilities[1],
            'label 2': probabilities[2],
            'label 3': probabilities[3],
            'label 4': probabilities[4],
            'label 5': probabilities[5]
        })

print(f"Predictions saved to {output_file}")

Predictions saved to /content/drive/MyDrive/3c-citation-text-classification/scisum/data/predictions.csv
